In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import os
import requests
import tensorflow_hub as hub
import time
import bert
import tensorflow
import matplotlib as plt

from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
from xgboost import XGBClassifier
from tqdm import tqdm
from time import sleep
from matplotlib.figure import Figure
from PIL import Image

from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from tensorflow.keras import layers, Sequential, callbacks, optimizers, utils, models, applications
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

2023-08-09 20:17:07.564620: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 20:17:07.908356: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 20:17:07.911224: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 20:17:09.743744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [95]:
!pip install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [109]:
folder_path = '/home/jessica/code/chooj202/movie_genre_prediction/raw_data/csv/jess csv'
dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)
merged_df = pd.concat(dataframes, ignore_index=True)

In [110]:
merged_df = merged_df.drop(columns = ['Unnamed: 0', 'movie', 'image_url', 'plot'])

In [111]:
merged_df = merged_df.astype(str).drop_duplicates()

In [112]:
merged_df

,imdb_id,genre
0,tt0075143,"['Action', ' Adventure', ' Biography']"
1,tt0120815,"['Drama', ' War']"
2,tt0118799,"['Comedy', ' Drama', ' Romance']"
3,tt0116860,"['Comedy', ' Drama', ' History']"
4,tt0253474,"['Biography', ' Drama', ' Music']"
...,...,...
1697,tt1073241,"['Crime', ' Drama', ' Mystery']"
1698,tt0034398,"['Horror', ' Mystery', ' Romance']"
1699,tt0478134,"['Crime', ' Drama', ' Mystery']"
1700,tt1787988,"['Crime', ' Drama', ' Mystery']"


In [113]:
merged_df["genre"] = (
    merged_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [114]:
merged_df = merged_df.dropna()

In [115]:
merged_df.head()

,imdb_id,genre
0,tt0075143,"[Action, Adventure, Biography]"
1,tt0120815,"[Drama, War]"
2,tt0118799,"[Comedy, Drama, Romance]"
3,tt0116860,"[Comedy, Drama, History]"
4,tt0253474,"[Biography, Drama, Music]"


In [116]:
merged_df

,imdb_id,genre
0,tt0075143,"[Action, Adventure, Biography]"
1,tt0120815,"[Drama, War]"
2,tt0118799,"[Comedy, Drama, Romance]"
3,tt0116860,"[Comedy, Drama, History]"
4,tt0253474,"[Biography, Drama, Music]"
...,...,...
1697,tt1073241,"[Crime, Drama, Mystery]"
1698,tt0034398,"[Horror, Mystery, Romance]"
1699,tt0478134,"[Crime, Drama, Mystery]"
1700,tt1787988,"[Crime, Drama, Mystery]"


In [117]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(merged_df['genre'])

#transform target variable
y = multilabel_binarizer.transform(merged_df['genre'])
genre_names = multilabel_binarizer.classes_

# Adding 
for i in range(len(genre_names)):
    merged_df[f"{genre_names[i]}"] = y[:,i]

In [119]:
y.shape

(1398, 21)

In [120]:
np.save('y_array', y)

In [118]:
merged_df.to_csv("jess_small_dataset.csv")

In [96]:
# # resize all images to 256 x 256
# # convert images to arrays (ignore corrupted images)
# width, height = 256, 256
# image_array = []

# for i in tqdm(range(merged_df.shape[0])):
#     try:
#         image_path = f"/home/jessica/code/chooj202/movie_genre_prediction/raw_data/posters/{merged_df['imdb_id'][i]}.jpg"
#         img = Image.open(image_path)
#         input_arr = image.img_to_array(img)
#         input_arr = input_arr/255.0
#         image_array.append([merged_df['imdb_id'][i], input_arr])
#         time.sleep(1)
#     except:
#         pass

# img_array_df = pd.DataFrame(image_array, columns=["imdb_id", "image_array"])

100%|█████████████████████████████████████████████████████████████████████████████| 1702/1702 [00:00<00:00, 7787.25it/s]


In [10]:
imgs = []
images_path = f"/home/jessica/code/chooj202/movie_genre_prediction/raw_data/posters/{merged_df['imdb_id'][i]}.jpg"
for img in tqdm(range(merged_df.shape[0])):
    image = Image.open(images_path)
    image = image.resize((256, 256))
    imgs.append(np.array(image))
    time.sleep(1)

100%|███████████████████████████████████████████████████████████████████████████████| 1398/1398 [24:41<00:00,  1.06s/it]


In [121]:
X = np.array(imgs)

np.save('X_array', X)

In [68]:
new_df = pd.merge(img_array_df, merged_df, how = 'right', on = 'imdb_id')

In [71]:
new_df

,imdb_id,image_array,plot,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0075143,[[[0.99607843 0.99607843 1. ]\n [0.992...,"The story of prophet ""Muhammad"" and the delive...",1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0120815,[[[0.94509804 0.9254902 0.9490196 ]\n [0.819...,Opening with the Allied invasion of Normandy o...,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
5,tt0118799,[[[0.98039216 0.95686275 0.8627451 ]\n [0.980...,"In 1930s Italy, a carefree Jewish waiter named...",0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
8,tt0116860,[[[0.9647059 0.96862745 0.9843137 ]\n [0.988...,"At the Belgrade army hospital, casualties of B...",0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
10,tt0253474,[[[0.5254902 0.5254902 0.5254902 ]\n [0.325...,"In this adaptation of the autobiography ""The P...",0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,tt1073241,[[[0.23921569 0.21960784 0.23529412]\n [0.415...,Thinking Pulitzer Prize and hoping to bring do...,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
2392,tt0034398,[[[0.11764706 0.11764706 0.11764706]\n [0.105...,"Upon the death of his brother, Larry Talbot re...",0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,0
2393,tt0478134,[[[0.20784314 0.21176471 0.23137255]\n [0.117...,A retired military investigator works with a p...,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
2394,tt1787988,[[[0.8352941 0.40784314 0.00392157]\n [0.831...,Seeing reflections of Mumbai under the red lig...,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0


In [11]:
def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize
    stop_words = set(stopwords.words('english')) ## define stopwords
    
    tokenized_sentence_cleaned = [ ## remove stopwords
    w for w in tokenized_sentence if not w in stop_words]
    
    lemmatized = [
    WordNetLemmatizer().lemmatize(word, pos = "v")
        for word in tokenized_sentence_cleaned
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence

In [72]:
new_df['plot'] = new_df['plot'].apply(cleaning)

In [73]:
new_df = pd.DataFrame(new_df)

In [75]:
new_df.to_csv("new_df.csv")

In [23]:
new_df.head()

,imdb_id,image_array,plot,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0075143,[[[0.9882353 0.9882353 0.99607843]\n [0.992...,story prophet muhammad delivery message god allah,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0120815,[[[0.94509804 0.9254902 0.9490196 ]\n [0.764...,open ally invasion normandy june members nd ra...,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
5,tt0118799,[[[0.98039216 0.95686275 0.8627451 ]\n [0.980...,italy carefree jewish waiter name guido start ...,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
8,tt0116860,[[[0.9647059 0.96862745 0.9843137 ]\n [0.988...,belgrade army hospital casualties bosnian civi...,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
10,tt0253474,[[[0.5254902 0.5254902 0.5254902 ]\n [0.325...,adaptation autobiography pianist extraordinary...,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [90]:
X = np.array(imgs)

In [91]:
X.shape

(1398, 256, 256, 3)

In [14]:
merged_df = merged_df.drop(columns = ['imdb_id', 'plot'])

In [92]:
y.shape

(248, 21)

In [15]:
y = merged_df
y.to_csv("Y.csv")

In [2]:
X = np.load('X_array.npy')
y = np.load('y_array.npy')

In [3]:
X_train, X_test, y_train ,y_test = train_test_split(X, y, test_size=0.2)

In [4]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5)

In [11]:
X_train.shape

(1118, 256, 256, 3)

In [7]:
X_val.shape

(140, 256, 256, 3)

In [8]:
X_test.shape

(140, 256, 256, 3)

In [9]:
y_test.shape

(140, 21)

In [10]:
y_val.shape

(140, 21)

In [11]:
y_train.shape

(1118, 21)

## CNN

In [15]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def load_own_model():

    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(256,256,3)))

    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(21, activation='softmax'))
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

In [16]:
model_homemade = load_own_model()
model_homemade.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 247, 247, 16)      4816      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 82, 82, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 75, 75, 32)        32800     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 25, 25, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 20, 20, 32)       

In [17]:
es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)

history = model_homemade.fit(X_train, y_train,
                             validation_data = (X_val, y_val),
                             batch_size = 16, 
                             epochs = 100, 
                             callbacks=[es])

Epoch 1/100
70/70 [==============================] - 60s 844ms/step - loss: 49.7970 - accuracy: 0.2639 - val_loss: 269.3107 - val_accuracy: 0.3429
Epoch 2/100
70/70 [==============================] - 59s 838ms/step - loss: 3067.3442 - accuracy: 0.2174 - val_loss: 10509.1406 - val_accuracy: 0.0000e+00
Epoch 3/100
70/70 [==============================] - 60s 854ms/step - loss: 35754.5547 - accuracy: 0.1950 - val_loss: 84658.4062 - val_accuracy: 0.3429
Epoch 4/100
70/70 [==============================] - 59s 843ms/step - loss: 185503.0938 - accuracy: 0.2021 - val_loss: 338835.7812 - val_accuracy: 0.2143
Epoch 5/100
70/70 [==============================] - 60s 854ms/step - loss: 540912.0000 - accuracy: 0.2147 - val_loss: 923525.9375 - val_accuracy: 0.1571
Epoch 6/100
70/70 [==============================] - 59s 844ms/step - loss: 1286148.0000 - accuracy: 0.2030 - val_loss: 1910037.0000 - val_accuracy: 0.1571
Epoch 6: early stopping


In [18]:
res = model_homemade.evaluate(X_test, y_test)

5/5 [==============================] - 1s 125ms/step - loss: 285.3484 - accuracy: 0.2786


In [19]:
test_accuracy = res[-1]
print(f"test_accuracy = {round(test_accuracy,2)*100} %")

test_accuracy = 28.000000000000004 %


## RESNET 50

In [2]:
!pip3 install nvidia-tensorrt

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1-py2.py3-none-any.whl size=16972 sha256=0763287a6dc9951ce1b79810c204573546288c31f966df08ce3b978153c65d4a
  Stored in directory: /home/jessica/.cache/pip/wheels/6d/29/56/abdffd4c604f255b5254bef3f1c598ab7811ea020540599438
Successfully built tensorrt


In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50

def load_model():
    
    # $CHALLENGIFY_BEGIN
    
    model = ResNet50(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
    
    # $CHALLENGIFY_END
    
    return model

model = load_model()

In [6]:
def set_nontrainable_layers(model):
    
    # $CHALLENGIFY_BEGIN
    # Set the first layers to be untrainable
    model.trainable = False
    
    # $CHALLENGIFY_END
    
    return model

model = set_nontrainable_layers(model)
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [7]:
from tensorflow.keras import layers, models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    # $CHALLENGIFY_BEGIN
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(21, activation='softmax')
    
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    # $CHALLENGIFY_END
    return model

model = add_last_layers(model)

In [8]:
from tensorflow.keras import optimizers

def build_model():
    # $CHALLENGIFY_BEGIN    
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model
    # $CHALLENGIFY_END

model = build_model()

In [9]:
from tensorflow.keras.applications.resnet50 import preprocess_input

X_train = preprocess_input(X_train) 
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)

history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=50, 
                    batch_size=16, 
                    callbacks=[es])

Epoch 1/50


2023-08-09 20:21:35.102849: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 879230976 exceeds 10% of free system memory.


16/70 [=====>........................] - ETA: 1:14 - loss: 45.5303 - accuracy: 0.2070

In [ ]:
res_resnet = model.evaluate(X_test, y_test)

test_accuracy_resnet = res_resnet[-1]


print(f"test_accuracy_resnet = {round(test_accuracy_resnet,2)*100} %")

print(f"test_accuracy = {round(test_accuracy,2)*100} %")

## VGG 19

## ResNet 50

## Stacking Multimodal Model

In [67]:
tk = Tokenizer()
tk.fit_on_texts(X_train['plot'])
vocab_size = len(tk.word_index)
print(f'There are {vocab_size} different words in your corpus')
X_token = tk.texts_to_sequences(X_train['plot'])

X_pad = pad_sequences(X_token, dtype = 'float32', padding = 'post')

There are 1582 different words in your corpus


In [68]:
vocab_size

1582

In [ ]:
multilabel_binarizer.classes_.shape

In [79]:
# define two sets of inputs
inputA = layers.Input(shape = (350,350,3))
inputB = layers.Input(shape=(352,352,3))

# the first branch operates on the first input
x = layers.Embedding(
    input_dim = vocab_size +1,
    input_length = 292,
    output_dim = 100,
    mask_zero = True)
x = layers.Dense(1024, activation = 'relu')(inputA)
x = layers.Dense(21, activation='softmax')(x)
x = models.Model(inputs = inputA, outputs=x)

# the second branch operates on the second input
y = layers.Conv2D(16, kernel_size = (3,3), activation="relu")(inputB)
y = layers.Dense(32, activation="relu")(y)
y = layers.Dense(21, activation="softmax")(y)
y = models.Model(inputs = inputB, outputs=y)

# combine the output of the two branches
combined = layers.concatenate([x.output, y.output])

# apply a FC layer and then a regression prediction on the
# combined outputs

z = layers.Dense(10, activation="relu")(combined)
z = layers.Dense(21, activation="softmax")(z)

# our model will accept the inputs of the two branches and
# then output a single value
model = models.Model(inputs=[x.input, y.input], outputs=z)

In [ ]:
input_word_ids = layers.Input(shape=(max_seq_length,), dtype=tensorflow.int32,
                                           name="input_word_ids")
input_mask = layers.Input(shape=(max_seq_length,), dtype=tensorflow.int32,
                                       name="input_mask")
segment_ids = layers.Input(shape=(max_seq_length,), dtype=tensorflow.int32,
                                        name="segment_ids")

seq_out = bert_layer([input_ids, input_masks, input_segments])
out = layers.LSTM(128, name='LSTM')(seq_out)
model_lstm = models.Model([input_ids, input_masks, input_segments], out)

In [ ]:
s = "This is a nice sentence."
stokens = tokenizer.tokenize(s)
stokens = ["[CLS]"] + stokens + ["[SEP]"]

input_ids = get_ids(stokens, tokenizer, max_seq_length)
input_masks = get_masks(stokens, max_seq_length)
input_segments = get_segments(stokens, max_seq_length)

pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])